<a href="https://colab.research.google.com/github/terkoizmy/TheSocialDilemma/blob/main/Sentiment_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Transformers

In [37]:
!pip install transformers

# Import Library

In [38]:
import torch
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification

dataset = pd.read_csv('TheSocialDilemma.csv')
dataset

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,Sentiment
0,Mari Smith,"San Diego, California",Premier Facebook Marketing Expert | Social Med...,2007-09-11 22:22:51,579942,288625,11610,False,2020-09-16 20:55:33,@musicmadmarc @SocialDilemma_ @netflix @Facebo...,NaN,Twitter Web App,False,Neutral
1,Mari Smith,"San Diego, California",Premier Facebook Marketing Expert | Social Med...,2007-09-11 22:22:51,579942,288625,11610,False,2020-09-16 20:53:17,@musicmadmarc @SocialDilemma_ @netflix @Facebo...,NaN,Twitter Web App,False,Neutral
2,Varun Tyagi,"Goa, India",Indian | Tech Solution Artist & Hospitality Ex...,2009-09-06 10:36:01,257,204,475,False,2020-09-16 20:51:57,Go watch “The Social Dilemma” on Netflix!\n\nI...,NaN,Twitter for iPhone,False,Positive
3,Casey Conway,"Sydney, New South Wales",Head of Diversity & Inclusion @RugbyAU | It's ...,2012-12-28 21:45:06,11782,1033,12219,True,2020-09-16 20:51:46,I watched #TheSocialDilemma last night. I’m sc...,['TheSocialDilemma'],Twitter for iPhone,False,Negative
4,Charlotte Paul,Darlington,Instagram Charlottejyates,2012-05-28 20:43:08,278,387,5850,False,2020-09-16 20:51:11,The problem of me being on my phone most the t...,['TheSocialDilemma'],Twitter for iPhone,False,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20063,scp.,NaN,"“Through love, all is possible.” - SJM - See m...",2013-02-19 00:55:12,431,193,32958,False,2020-10-09 00:25:53,#TheSocialDilemma yalll.... this shit... we kn...,['TheSocialDilemma'],Twitter for iPhone,False,Negative
20064,Dono6971,United States,"Father, Husband, and a Dude|| Love Notre Dame ...",2010-01-06 04:08:41,172,96,50159,False,2020-10-09 00:24:45,Peeps:\n\nFind 90 minutes this weekend and wat...,NaN,Twitter for iPhone,False,Positive
20065,Remi Shores,NaN,Genderfluid / They/Them/Theirs / Queer Christi...,2012-05-16 23:49:13,387,652,7885,False,2020-10-09 00:11:42,"So you watched #thesocialdilemma, or have been...",['thesocialdilemma'],Twitter Web App,False,Negative
20066,Scott the Great and Terrible,NaN,"I can't recall the taste of food, nor the soun...",2020-03-16 18:20:31,103,84,2976,False,2020-10-09 00:10:16,Good social media advice:\n\nChoose the thing ...,['TheSocialDilemma'],Twitter Web App,False,Positive


# Mount Gdrive

  if you practice this project in Colab you need save your model in drive

In [39]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Data Processing

In [40]:
df = dataset.iloc[:,[9,13]]

In [41]:
df

,text,Sentiment
0,@musicmadmarc @SocialDilemma_ @netflix @Facebo...,Neutral
1,@musicmadmarc @SocialDilemma_ @netflix @Facebo...,Neutral
2,Go watch “The Social Dilemma” on Netflix!\n\nI...,Positive
3,I watched #TheSocialDilemma last night. I’m sc...,Negative
4,The problem of me being on my phone most the t...,Positive
...,...,...
20063,#TheSocialDilemma yalll.... this shit... we kn...,Negative
20064,Peeps:\n\nFind 90 minutes this weekend and wat...,Positive
20065,"So you watched #thesocialdilemma, or have been...",Negative
20066,Good social media advice:\n\nChoose the thing ...,Positive


In [42]:
df['Sentiment'].value_counts()

Positive    9513
Neutral     6982
Negative    3573
Name: Sentiment, dtype: int64

In [43]:
possible_labels = df.Sentiment.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Negative': 2, 'Neutral': 0, 'Positive': 1}

In [44]:
df['label'] = df.Sentiment.replace(label_dict)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [45]:
df

,text,Sentiment,label
0,@musicmadmarc @SocialDilemma_ @netflix @Facebo...,Neutral,0
1,@musicmadmarc @SocialDilemma_ @netflix @Facebo...,Neutral,0
2,Go watch “The Social Dilemma” on Netflix!\n\nI...,Positive,1
3,I watched #TheSocialDilemma last night. I’m sc...,Negative,2
4,The problem of me being on my phone most the t...,Positive,1
...,...,...,...
20063,#TheSocialDilemma yalll.... this shit... we kn...,Negative,2
20064,Peeps:\n\nFind 90 minutes this weekend and wat...,Positive,1
20065,"So you watched #thesocialdilemma, or have been...",Negative,2
20066,Good social media advice:\n\nChoose the thing ...,Positive,1


## Spliting Data

  Becouse label in this data is imbalance we split the data set in a stratified fashion, using this as the class labels.

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(df["text"].values,
                                                  df["label"].values, 
                                                  test_size=0.30, 
                                                  random_state=42)

## Encode and Tokenizer data

Tokenization is a process to take raw texts and split into tokens, which are numeric data to represent words.

In [47]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    X_train, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    X_val, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


## Transform data to tensor

  for the bert model we need change the array to tensor data for a better modeling

In [48]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(y_train)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(y_val)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

# BERT Pre-trained Model

We are treating each title as its unique sequence, so one sequence will be classified to one of the five labels

In [49]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Data Loaders

we combines a dataset and a sampler, and provides an iterable over the given dataset

In [50]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 3
dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

## Optimizer & scheduler

To construct an optimizer, we have to give it an iterable containing the parameters to optimize. Then, we can specify optimizer-specific options such as the learning rate, epsilon, etc.

In [51]:
from transformers import AdamW, get_linear_schedule_with_warmup
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)        
epochs = 5
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

## Performance Metrics

We will use f1 score and accuracy per class as performance metrics.

In [52]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

# Training loop data

In [53]:
import random
device = torch.device("cuda")
model = model.to(device)
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    
    torch.save(model.state_dict(), F'/content/gdrive/MyDrive/BertModel/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(F'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(F'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(F'Validation loss: {val_loss}')
    tqdm.write(F'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.5706534041572575
Validation loss: 0.4338788764888173
F1 Score (Weighted): 0.9202635545655824



Epoch 2
Training loss: 0.29820408601595083
Validation loss: 0.3848289226444827
F1 Score (Weighted): 0.9308459388281946



Epoch 3
Training loss: 0.17561361420324675
Validation loss: 0.3549872048212054
F1 Score (Weighted): 0.9431057317875637



Epoch 4
Training loss: 0.10842021062335278
Validation loss: 0.40224553299735416
F1 Score (Weighted): 0.9406765126545362



Epoch 5
Training loss: 0.07068400632319383
Validation loss: 0.3937095775627245
F1 Score (Weighted): 0.946017982808519



# Loading and Evaluating the Model

In [55]:
#device = torch.device("cuda") ## if you all ready save a model you need repeat this code
#model = model.to(device) ##if you all ready save a model you need repeat this code

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('/content/gdrive/MyDrive/BertModel/finetuned_BERT_epoch_5.model', map_location=torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Class: Neutral
Accuracy: 1947/2049

Class: Positive
Accuracy: 2759/2869

Class: Negative
Accuracy: 990/1103



# Confusion Matrix

In [63]:
predict = np.argmax(predictions, axis=1).flatten()
data = {'y_Actual':    true_vals,
        'y_Predicted': preds_flat1
        }

df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])

confusion_matrix = pd.crosstab(df['y_Actual'], df['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix)

Predicted     0     1    2
Actual                    
0          1947    56   46
1            43  2759   67
2            36    77  990


* Precission{Neutral = 0.96, Positive = 0.95, Negative = 0.897}
* Recall{Neutral = 0.95, Positive = 0.96, Negative = 0.89}
* Accuracy = 0.94

In [66]:
label_dict

{'Negative': 2, 'Neutral': 0, 'Positive': 1}